# data1

In [1]:
clear;
close all;
clc;

In [2]:
symbol ='SP500';
endDate = datenum('Dec-31-2017');
c = fred;
d = fetch(c,symbol,endDate-3652,endDate);
time = datetime(d.Data(:,1),'ConvertFrom','datenum');
prices = d.Data(:,2);

In [3]:
T = timetable(time,prices);
clearvars -except T
save('data1.mat');

# data2

In [4]:
clear;
close all;
clc;
load data1.mat

In [15]:
T = rmmissing(T);

In [16]:
[T.y,T.m,T.d] = ymd(T.time);

In [17]:
T.dayOfWeek = day(T.time,'dayofweek');

In [18]:
dayDiff = diff(datenum(T.time));
T.nNextTD = [dayDiff; NaN];
T.nPreviousTD = [NaN; dayDiff];

In [20]:
nDay1 = [1, 5, 10, 20];

In [22]:
for i = 1:numel(nDay1)
    factorName = ['ret' num2str(nDay1(i))];
    T.(factorName) = nDayReturn(T.prices, nDay1(i));
end

Error using eval
Undefined function or variable 'nDayReturn'.



In [23]:
nDay2 = [7, 14, 21];
for i = 1:numel(nDay2)
    factorName = ['rsi' num2str(nDay2(i))];
    T.(factorName) = rsindex(T.prices,nDay2(i));
end

In [24]:
T.MACD = macd(T.prices);

In [25]:
nextDayReturn = double(T.ret1(2:end) > 0);
response = categorical(nextDayReturn,[1,0],{'Buy','Sell'});
title('Distribution of Response Variable');

Unrecognized variable name 'ret1'.



In [26]:
clearvars -except T

In [27]:
T = rmmissing(T);
T = timetable2table(T);
T.time = []; % Remove time
T.prices = []; % Remove prices
x = T{:,1:end-1};
y = T{:,end};
save('data2.mat')

In [28]:
function returns = nDayReturn(prices, nDay)
% This function is to generate nDay return give the price vector
returns = NaN(size(prices,1),1);
returns(nDay+1:end) = prices(nDay+1:end) ./ prices(1:end-nDay) - 1;
end

function rsi = nDayRSI(prices, nDay)
% This function is to generate nDay RSI give the price vector
returns = rsindex(prices,nDay);
returns(nDay+1:end) = prices(nDay+1:end) ./ prices(1:end-nDay) - 1;
end

Error: Function definition not supported in this context. Create functions in code file.



# data3

In [29]:
clear;
close all;
clc;
rng(0); % Fix random seed
load data2.mat

In [30]:
nOut = 252*2;

In [31]:
N = height(T);
nIn = N - nOut;
idxIn = [1:nIn]';
idxOut = [nIn + 1:N]';

In [32]:
inSample = T(idxIn,:);
outSample = T(idxOut,:);

In [33]:
yRetInSample = T.ret1(idxIn);
yRetOutSample = T.ret1(idxOut);
save('data3_demo1.mat', 'inSample','outSample',...
    'yRetInSample','yRetOutSample')

Error using tabular/dotParenReference (line 69)
Unrecognized variable name 'ret1'.



In [34]:
nBuy = sum(inSample.response == 'Buy')

Unrecognized variable name 'response'.

